In [6]:
import numpy as np

def ista_solve_hot( A, d, la_array ):
    # ista_solve_hot: Iterative soft-thresholding for multiple values of
    # lambda with hot start for each case - the converged value for the previous
    # value of lambda is used as an initial condition for the current lambda.
    # this function solves the minimization problem
    # Minimize |Ax-d|_2^2 + lambda*|x|_1 (Lasso regression)
    # using iterative soft-thresholding.
    max_iter = 10**4
    tol = 10**(-3)
    tau = 1/np.linalg.norm(A,2)**2
    n = A.shape[1]
    w = np.zeros((n,1))
    num_lam = len(la_array)
    X = np.zeros((n, num_lam))
    for i, each_lambda in enumerate(la_array):
        for j in range(max_iter):
            z = w - tau*(A.T@(A@w-d))
            w_old = w
            w = np.sign(z) * np.clip(np.abs(z)-tau*each_lambda/2, 0, np.inf)
            X[:, i:i+1] = w
            if np.linalg.norm(w - w_old) < tol:
                break
    return X


In [7]:
## Breast Cancer LASSO Exploration
## Prepare workspace
from scipy.io import loadmat
from sklearn import linear_model

import numpy as np
X = loadmat("BreastCancer.mat")['X']
y = loadmat("BreastCancer.mat")['y']

##  10-fold CV 

# each row of setindices denotes the starting an ending index for one
# partition of the data: 5 sets of 30 samples and 5 sets of 29 samples
setindices = [[1,30],[31,60],[61,90],[91,120],[121,150],[151,179],[180,208],[209,237],[238,266],[267,295]]

# each row of holdoutindices denotes the partitions that are held out from
# the training set
holdoutindices = [[1,2],[2,3],[3,4],[4,5],[5,6],[7,8],[9,10],[10,1]]

W = []
lam_values = [3.5, 4, 4.5,5,5.5,6,6.5,7,7.5]
cases = len(holdoutindices)

# be sure to initiate the quantities you want to measure before looping
# through the various training, validation, and test partitions
#
# 
#
errors = []
# Loop over various cases
for j in range(cases):
    # row indices of first validation set
    v1_ind = np.arange(setindices[holdoutindices[j][0]-1][0]-1,setindices[holdoutindices[j][0]-1][1])
    
    # row indices of second validation set
    v2_ind = np.arange(setindices[holdoutindices[j][1]-1][0]-1,setindices[holdoutindices[j][1]-1][1])
    
    # row indices of training set
    trn_ind = list(set(range(295))-set(v1_ind)-set(v2_ind))
    
    # define matrix of features and labels corresponding to first
    # validation set
    Av1 = X[v1_ind,:]
    bv1 = y[v1_ind]
    
    # define matrix of features and labels corresponding to second
    # validation set
    Av2 = X[v2_ind,:]
    bv2 = y[v2_ind]
    
    # define matrix of features and labels corresponding to the 
    # training set
    At = X[trn_ind,:]
    bt = y[trn_ind]

    # clf = linear_model.Lasso()
    # clf.fit()
    # print(clf.coef_)

    w = ista_solve_hot(At,bt,lam_values)

    errors.append([np.mean(np.abs(Av1@i - bv1)) for i in w.T])

    W.append(w)

    print(len(v1_ind), len(v2_ind), len(trn_ind))
# Use training data to learn classifier
#
# Find best lambda value using first validation set, then evaluate
# performance on second validation set, and accumulate performance metrics
# over all cases partitions
cases

30 30 235
30 30 235
30 30 235
30 30 235
30 29 236
29 29 237
29 29 237
29 30 236


8

In [8]:
np.eye(5)

array([[1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1.]])

In [9]:
min_i = None
min_j = None
min_error = None
for i in range(len(errors)):
    for j in range(len(errors[i])):
        if min_error == None or min_error >= errors[i][j]:
            min_i = i
            min_j = j
            min_error = errors[i][j]
min_i , min_j

(7, 8)

In [10]:
holdoutindices[min_i]

v1_ind = np.arange(setindices[holdoutindices[min_i][0]-1][0]-1,setindices[holdoutindices[min_i][0]-1][1])
    
    # row indices of second validation set
v2_ind = np.arange(setindices[holdoutindices[min_i][1]-1][0]-1,setindices[holdoutindices[min_i][1]-1][1])
    

trn_ind = list(set(range(295))-set(v1_ind)-set(v2_ind))

    # define matrix of features and labels corresponding to first
    # validation set
Av1 = X[v1_ind,:]
bv1 = y[v1_ind]
    
    # define matrix of features and labels corresponding to second
    # validation set
Av2 = X[v2_ind,:]
bv2 = y[v2_ind]

At = X[trn_ind,:]
bt = y[trn_ind]

    # clf = linear_model.Lasso()
    # clf.fit()
    # print(clf.coef_)

w = ista_solve_hot(At,bt,[lam_values[min_j]])


y_hat_1 = [np.sign(Av1@i) - bv1 for i in w.T] 
y_hat_2 = [np.sign(Av2@i) - bv2 for i in w.T] 
y_hat_1 = np.squeeze(np.asarray(y_hat_1[0]))
y_hat_2 = np.squeeze(np.asarray(y_hat_2[0]))
misclassifications = 0
total = 0
for i in y_hat_1:
    for j in i:
        if j != 0:
            misclassifications += 1
        total += 1

for i in y_hat_2:
    for j in i:
        if j != 0:
            misclassifications += 1
        total += 1

misclassifications = misclassifications / total

residual = [np.linalg.norm(Av1@i - bv1) for i in w.T][0] + [np.linalg.norm(Av2@i - bv2) for i in w.T][0]

print(f"Error rate (Lasso) = {misclassifications}")
print(f"Residual error (Lasso) = {residual}")

Error rate = 0.41929925330269957
Residual error = 64.48843268972031


In [12]:
w_ridge = np.linalg.inv(At.T @ At + (lam_values[min_j] * np.eye(len(At.T @ At)))) 
w_ridge = w_ridge @ At.T 
w_ridge = w_ridge @ bt
w_ridge

array([[-0.00530127],
       [ 0.00781243],
       [ 0.01010464],
       ...,
       [-0.00312042],
       [-0.00947405],
       [-0.00375704]])

In [13]:
y_hat_1 = [np.sign(Av1@i) - bv1 for i in w_ridge.T] 
y_hat_2 = [np.sign(Av2@i) - bv2 for i in w_ridge.T] 
y_hat_1 = np.squeeze(np.asarray(y_hat_1[0]))
y_hat_2 = np.squeeze(np.asarray(y_hat_2[0]))
misclassifications = 0
total = 0
for i in y_hat_1:
    for j in i:
        if j != 0:
            misclassifications += 1
        total += 1

for i in y_hat_2:
    for j in i:
        if j != 0:
            misclassifications += 1
        total += 1

misclassifications = misclassifications / total

residual = [np.linalg.norm(Av1@i - bv1) for i in w_ridge.T][0] + [np.linalg.norm(Av2@i - bv2) for i in w_ridge.T][0]

print(f"Error rate (Ridge) = {misclassifications}")
print(f"Residual error (Ridge) = {residual}")

Error rate (Ridge) = 0.4330844342331993
Residual error (Ridge) = 67.60349496870052
